# Backtest Results

## Overview

This notebook performs comprehensive backtesting of multiple portfolio strategies:
1. **Baseline:** SPY buy-and-hold
2. **Equal-Weight:** 1/N allocation across all assets
3. **Volatility Parity:** Inverse-volatility weighting
4. **Risk Parity:** Equal risk contribution
5. **Mean-Variance:** Markowitz optimization (max Sharpe)
6. **Macro-Adaptive:** Dynamic allocation based on macro regime

Backtesting period: January 2015 - November 2025  
Rebalancing: Monthly  
Transaction costs: 10 bps per rebalance

In [ ]:
# Import libraries
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from data_pipeline import DataPipeline
from macro_features import MacroFeatureEngine
from portfolio_optimizer import PortfolioOptimizer, PortfolioBacktester

warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

%matplotlib inline
%load_ext autoreload
%autoreload 2

## 1. Data Preparation

In [ ]:
# Initialize and run data pipeline
print("Step 1: Fetching and processing data...\n")

pipeline = DataPipeline(
    start_date="2015-01-01",
    end_date="2025-11-30"
)

prices, merged_data = pipeline.run_pipeline(save_to_csv=True, output_dir='../data')

print(f"\nData loaded: {len(merged_data)} trading days")
print(f"Date range: {merged_data.index.min()} to {merged_data.index.max()}")

In [ ]:
# Build features
print("Step 2: Engineering features...\n")

feature_engine = MacroFeatureEngine(merged_data)

asset_returns = [col for col in merged_data.columns if '_RETURN' in col]
price_cols = ['WTI', 'XLE', 'XOM', 'CVX', 'IEF', 'SHY', 'SPY']
macro_factors = ['FEDFUNDS', 'CPIAUCSL', 'VIXCLS', 'INDPRO', 'T10Y2Y']

features_data = feature_engine.build_all_features(
    asset_returns=asset_returns,
    price_cols=price_cols,
    macro_factors=macro_factors
)

features_data.to_csv('../data/features_data.csv')
print(f"\nFeatures created: {len(features_data.columns)} total columns")

In [ ]:
# Display summary statistics
print("\n" + "="*80)
print("ASSET RETURN STATISTICS (Annualized %)")
print("="*80)

return_cols = ['WTI_RETURN', 'XLE_RETURN', 'XOM_RETURN', 'CVX_RETURN', 'IEF_RETURN', 'SPY_RETURN']
summary = pd.DataFrame({
    'Mean': features_data[return_cols].mean() * 252 * 100,
    'Volatility': features_data[return_cols].std() * np.sqrt(252) * 100,
    'Sharpe': (features_data[return_cols].mean() / features_data[return_cols].std()) * np.sqrt(252),
    'Skewness': features_data[return_cols].skew(),
    'Kurtosis': features_data[return_cols].kurtosis()
})

print(summary.round(2))

## 2. Portfolio Backtesting

In [ ]:
# Prepare data for backtesting
print("Step 3: Preparing portfolio data...\n")

# Select assets for the ETF
# Core holdings: Energy (WTI, XLE, XOM, CVX) + Diversifier (IEF)
portfolio_assets = ['WTI_RETURN', 'XLE_RETURN', 'XOM_RETURN', 'CVX_RETURN', 'IEF_RETURN']
portfolio_returns = features_data[portfolio_assets].copy()
portfolio_returns.columns = ['WTI', 'XLE', 'XOM', 'CVX', 'IEF']
portfolio_returns = portfolio_returns.dropna()

# SPY benchmark
spy_returns = features_data['SPY_RETURN'].dropna()

# Macro scores
macro_scores = features_data['MACRO_SCORE'].reindex(portfolio_returns.index).fillna(0)

print(f"Portfolio assets: {list(portfolio_returns.columns)}")
print(f"Backtest period: {portfolio_returns.index.min()} to {portfolio_returns.index.max()}")
print(f"Number of trading days: {len(portfolio_returns)}")

In [ ]:
# Initialize backtester
backtester = PortfolioBacktester(
    portfolio_returns,
    rebalance_freq='M',  # Monthly rebalancing
    transaction_cost=0.001  # 10 bps
)

# Initialize optimizer
optimizer = PortfolioOptimizer(portfolio_returns, risk_free_rate=0.02)

print("\nBacktester initialized.")
print(f"Rebalancing frequency: Monthly")
print(f"Transaction cost: 10 bps per rebalance")

In [ ]:
# Run backtests for all strategies
print("\nStep 4: Running backtests...\n")
print("="*80)

strategies = {}

# 0. SPY Benchmark (buy and hold)
print("\n[0/6] Benchmark: SPY Buy-and-Hold")
spy_value = (1 + spy_returns.reindex(portfolio_returns.index).fillna(0)).cumprod()
strategies['SPY_Benchmark'] = pd.DataFrame({
    'portfolio_return': spy_returns.reindex(portfolio_returns.index).fillna(0),
    'portfolio_value': spy_value
})

# 1. Equal Weight
print("\n[1/6] Equal Weight Strategy")
strategies['Equal_Weight'] = backtester.backtest_strategy(
    'Equal Weight',
    lambda: optimizer.equal_weight()
)

# 2. Volatility Parity
print("\n[2/6] Volatility Parity Strategy")
strategies['Volatility_Parity'] = backtester.backtest_strategy(
    'Volatility Parity',
    lambda: optimizer.volatility_parity(lookback=126)
)

# 3. Risk Parity
print("\n[3/6] Risk Parity Strategy")
strategies['Risk_Parity'] = backtester.backtest_strategy(
    'Risk Parity',
    lambda: optimizer.risk_parity(lookback=126)
)

# 4. Mean-Variance (Max Sharpe)
print("\n[4/6] Mean-Variance Optimization")
strategies['Mean_Variance'] = backtester.backtest_strategy(
    'Mean-Variance',
    lambda: optimizer.mean_variance_optimization(lookback=126)
)

# 5. Macro-Adaptive
print("\n[5/6] Macro-Adaptive Strategy")
strategies['Macro_Adaptive'] = backtester.backtest_strategy(
    'Macro-Adaptive',
    optimizer.macro_adaptive_weights,
    macro_scores=macro_scores,
    energy_indices=[0, 1, 2, 3],  # WTI, XLE, XOM, CVX
    safe_indices=[4],  # IEF
    tilt_magnitude=0.20
)

print("\n" + "="*80)
print("All backtests complete!")
print("="*80)

## 3. Performance Metrics

In [ ]:
# Calculate performance metrics for all strategies
print("\nStep 5: Calculating performance metrics...\n")

metrics_dict = {}

for name, results in strategies.items():
    metrics = backtester.calculate_performance_metrics(results, risk_free_rate=0.02)
    metrics_dict[name] = metrics

metrics_df = pd.DataFrame(metrics_dict).T

print("\n" + "="*80)
print("PERFORMANCE METRICS COMPARISON")
print("="*80)
print(metrics_df.round(2))

# Save metrics
metrics_df.to_csv('../data/performance_metrics.csv')
print("\nMetrics saved to: ../data/performance_metrics.csv")

In [ ]:
# Highlight best performers
print("\n" + "="*80)
print("BEST PERFORMING STRATEGIES")
print("="*80)

print(f"\nHighest Annual Return: {metrics_df['Annual Return (%)'].idxmax()}")
print(f"  Return: {metrics_df['Annual Return (%)'].max():.2f}%")

print(f"\nHighest Sharpe Ratio: {metrics_df['Sharpe Ratio'].idxmax()}")
print(f"  Sharpe: {metrics_df['Sharpe Ratio'].max():.2f}")

print(f"\nLowest Max Drawdown: {metrics_df['Max Drawdown (%)'].idxmax()}")
print(f"  Drawdown: {metrics_df['Max Drawdown (%)'].max():.2f}%")

print(f"\nHighest Sortino Ratio: {metrics_df['Sortino Ratio'].idxmax()}")
print(f"  Sortino: {metrics_df['Sortino Ratio'].max():.2f}")

## 4. Cumulative Performance Charts

In [ ]:
# Plot cumulative returns
fig, ax = plt.subplots(figsize=(14, 7))

for name, results in strategies.items():
    ax.plot(results.index, results['portfolio_value'], label=name.replace('_', ' '), linewidth=2)

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Portfolio Value ($)', fontsize=12)
ax.set_title('Cumulative Performance Comparison (Jan 2015 - Nov 2025)', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../data/cumulative_performance.png', dpi=300, bbox_inches='tight')
plt.show()

print("Chart saved to: ../data/cumulative_performance.png")

## 5. Rolling Metrics

In [ ]:
# Calculate rolling Sharpe ratios (1-year window)
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Rolling Sharpe
ax = axes[0]
for name, results in strategies.items():
    returns = results['portfolio_return']
    rolling_sharpe = (returns.rolling(252).mean() / returns.rolling(252).std()) * np.sqrt(252)
    ax.plot(results.index, rolling_sharpe, label=name.replace('_', ' '), linewidth=2)

ax.set_ylabel('Sharpe Ratio', fontsize=12)
ax.set_title('Rolling 1-Year Sharpe Ratio', fontsize=13, fontweight='bold')
ax.legend(loc='best', fontsize=9)
ax.grid(True, alpha=0.3)
ax.axhline(0, color='black', linestyle='--', linewidth=1)

# Rolling Volatility
ax = axes[1]
for name, results in strategies.items():
    returns = results['portfolio_return']
    rolling_vol = returns.rolling(252).std() * np.sqrt(252) * 100
    ax.plot(results.index, rolling_vol, label=name.replace('_', ' '), linewidth=2)

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Volatility (%)', fontsize=12)
ax.set_title('Rolling 1-Year Volatility (Annualized)', fontsize=13, fontweight='bold')
ax.legend(loc='best', fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/rolling_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

print("Chart saved to: ../data/rolling_metrics.png")

## 6. Drawdown Analysis

In [ ]:
# Plot drawdowns
fig, ax = plt.subplots(figsize=(14, 7))

for name, results in strategies.items():
    values = results['portfolio_value']
    cummax = values.cummax()
    drawdown = (values - cummax) / cummax * 100
    ax.plot(results.index, drawdown, label=name.replace('_', ' '), linewidth=2)

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Drawdown (%)', fontsize=12)
ax.set_title('Drawdown from Peak', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)
ax.fill_between(ax.get_xlim(), -50, 0, alpha=0.1, color='red')
plt.tight_layout()
plt.savefig('../data/drawdown_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("Chart saved to: ../data/drawdown_analysis.png")

## 7. Save All Results

In [ ]:
# Save individual strategy results
for name, results in strategies.items():
    filename = f'../data/backtest_{name.lower()}.csv'
    results.to_csv(filename)
    print(f"Saved: {filename}")

print("\n" + "="*80)
print("All results saved successfully!")
print("="*80)

## Summary

This notebook has completed:
- Data acquisition and feature engineering
- Backtesting of 6 portfolio strategies (2015-2025)
- Performance metrics calculation and comparison
- Visualization of cumulative returns, rolling metrics, and drawdowns

**Next Steps:**
- Review `results_visualization.ipynb` for additional analysis
- Analyze regime-specific performance
- Prepare final report with interpretation and discussion